## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("./Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Current Description,Rain (inchies),Snow (inchies),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saldanha,-33.0117,17.9442,50.05,clear sky,0.0,0.0,66,8,11.43,ZA,2021-07-24 19:51:57
1,1,Busselton,-33.6500,115.3333,56.08,overcast clouds,0.0,0.0,73,97,15.86,AU,2021-07-24 19:44:39
2,2,Taoudenni,22.6783,-3.9836,104.61,clear sky,0.0,0.0,7,3,9.42,ML,2021-07-24 19:51:57
3,3,Hami,42.8000,93.4500,87.03,clear sky,0.0,0.0,36,0,6.04,CN,2021-07-24 19:51:58
4,4,Husavik,66.0449,-17.3389,64.31,broken clouds,0.0,0.0,89,56,8.19,IS,2021-07-24 19:44:33


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head()

,City ID,City,Lat,Lng,Max Temp,Current Description,Rain (inchies),Snow (inchies),Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Hami,42.8000,93.4500,87.03,clear sky,0.00,0.00,36,0,6.04,CN,2021-07-24 19:51:58
7,7,Hilo,19.7297,-155.0900,81.63,light rain,0.19,0.19,79,75,0.00,US,2021-07-24 19:49:59
8,8,Beachwood,41.4645,-81.5087,83.52,few clouds,0.00,0.00,72,20,4.00,US,2021-07-24 19:51:59
9,9,Hithadhoo,-0.6000,73.0833,81.99,light rain,0.32,0.32,78,28,11.86,MV,2021-07-24 19:49:29
10,10,Celestun,20.8667,-90.4000,86.13,few clouds,0.00,0.00,68,17,14.05,MX,2021-07-24 19:51:59


In [5]:
# 4a. Determine if there are any empty rows.
# filtered_cities_df.count()

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df = filtered_cities_df.dropna()
# filtered_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hami,CN,87.03,clear sky,42.8000,93.4500,
7,Hilo,US,81.63,light rain,19.7297,-155.0900,
8,Beachwood,US,83.52,few clouds,41.4645,-81.5087,
9,Hithadhoo,MV,81.99,light rain,-0.6000,73.0833,
10,Celestun,MX,86.13,few clouds,20.8667,-90.4000,
15,Kapaa,US,82.38,light rain,22.0752,-159.3190,
16,Tutoia,BR,85.98,scattered clouds,-2.7619,-42.2744,
18,Kerema,PG,75.00,few clouds,-7.9631,145.7785,
20,Vila Velha,BR,76.96,scattered clouds,-20.3297,-40.2925,
31,Lufilufi,WS,76.78,light rain,-13.8745,-171.5986,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

## Using code from https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
## to replace empty spaces with NaNs
hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)

## Dropping rows witn NAN
clean_hotel_df = hotel_df.dropna()

In [10]:
clean_hotel_df.count()

City                   237
Country                237
Max Temp               237
Current Description    237
Lat                    237
Lng                    237
Hotel Name             237
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "./Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} at {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))